# Fine-tune masked ruBERT-tiny2

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModel
from datasets import Dataset
import evaluate

Загружаем отобранные предложения. Предложения отбираются в скрипте preperror.py

In [2]:
with open('FullDN.json', encoding="utf-8") as f:
    sentences = pd.read_json(f, orient='records')
sentences.tail()

,text,label
10621,Предупреждение действовало до семи утра.По инф...,0
10622,Украина заняла шестоеместо.\n,0
10623,"""Если есть паритет и достоинство, Тайвань очен...",0
10624,"""В соответствие с докладной запиской директора...",0
10625,"""В последующем все виды работ были оплачены, н...",0


In [3]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds = raw_ds.class_encode_column('label')
raw_ds.features

Stringifying the column:   0%|          | 0/10626 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10626 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['0', '1'], id=None)}

Datasets:

In [4]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [15]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "cointegrated/rubert-tiny2"
# model_checkpoint = "DeepPavlov/rubert-base-cased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [16]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> rubert-tiny2 number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> rubert-tiny2 number of parameters: 29M'
'>>> BERT number of parameters: 110M'


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
text = "ранее [MASK] привлекался к уголовной ответственности за вандализм и кражу"

In [19]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> ранее он привлекался к уголовной ответственности за вандализм и кражу'
'>>> ранее неоднократно привлекался к уголовной ответственности за вандализм и кражу'
'>>> ранее не привлекался к уголовной ответственности за вандализм и кражу'
'>>> ранее суд привлекался к уголовной ответственности за вандализм и кражу'
'>>> ранее был привлекался к уголовной ответственности за вандализм и кражу'


In [20]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# Use batched=True to activate fast multithreading!
tokenized_datasets = raw_ds.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/8500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2126 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 8500
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 2126
    })
})

In [21]:
tokenizer.model_max_length

2048

In [23]:
chunk_size = 128

In [24]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 10'
'>>> Review 1 length: 21'
'>>> Review 2 length: 26'


In [25]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 57'


In [26]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 57'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result